In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/poetry/Kanye_West.txt
/kaggle/input/poetry/johnny-cash.txt
/kaggle/input/poetry/kanye-west.txt
/kaggle/input/poetry/bruno-mars.txt
/kaggle/input/poetry/dickinson.txt
/kaggle/input/poetry/amy-winehouse.txt
/kaggle/input/poetry/blink-182.txt
/kaggle/input/poetry/paul-simon.txt
/kaggle/input/poetry/patti-smith.txt
/kaggle/input/poetry/bieber.txt
/kaggle/input/poetry/disney.txt
/kaggle/input/poetry/jimi-hendrix.txt
/kaggle/input/poetry/lin-manuel-miranda.txt
/kaggle/input/poetry/adele.txt
/kaggle/input/poetry/dj-khaled.txt
/kaggle/input/poetry/beatles.txt
/kaggle/input/poetry/r-kelly.txt
/kaggle/input/poetry/lady-gaga.txt
/kaggle/input/poetry/radiohead.txt
/kaggle/input/poetry/britney-spears.txt
/kaggle/input/poetry/alicia-keys.txt
/kaggle/input/poetry/rihanna.txt
/kaggle/input/poetry/joni-mitchell.txt
/kaggle/input/poetry/dolly-parton.txt
/kaggle/input/poetry/drake.txt
/kaggle/input/poetry/Lil_Wayne.txt
/kaggle/input/poetry/notorious_big.txt
/kaggle/input/poetry/eminem.txt
/

In [3]:
from datasets import load_dataset
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import tensorflow as tf
data_files = {
    "train": [
"/kaggle/input/poetry/bieber.txt"    ]
}
dataset = load_dataset("text", data_files=data_files)

In [4]:
print((dataset["train"][0]))

{'text': 'What do you mean?'}


In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [6]:
def tokenize_function(txt):
    return tokenizer(txt["text"], truncation=True, padding="max_length", max_length=128)

In [7]:
def gen():
    for row in tokenized_dataset["train"]:
        yield (
            {"input_ids": row["input_ids"], "attention_mask": row["attention_mask"]},
            row["input_ids"]
        )

In [8]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tf_dataset = tf.data.Dataset.from_generator(
    gen,
    output_signature=(
        {
            "input_ids": tf.TensorSpec(shape=(128,), dtype=tf.int32),
            "attention_mask": tf.TensorSpec(shape=(128,), dtype=tf.int32),
        },
        tf.TensorSpec(shape=(128,), dtype=tf.int32),
    )
).shuffle(1000).batch(4)

Map:   0%|          | 0/3715 [00:00<?, ? examples/s]

I0000 00:00:1745134429.547343      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745134429.548105      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [9]:
model = TFGPT2LMHeadModel.from_pretrained("gpt2")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5))

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [10]:
model.fit(tf_dataset, epochs=5, verbose=1)

Epoch 1/5


I0000 00:00:1745134468.941898     102 service.cc:148] XLA service 0x7a78bc3038b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745134468.942996     102 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745134468.943015     102 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745134469.034689     102 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745134469.158929     102 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


929/929 [==============================] - 217s 186ms/step - loss: 0.2932
Epoch 2/5
929/929 [==============================] - 182s 195ms/step - loss: 0.1959
Epoch 3/5
929/929 [==============================] - 182s 195ms/step - loss: 0.1512
Epoch 4/5
929/929 [==============================] - 182s 195ms/step - loss: 0.1180
Epoch 5/5
929/929 [==============================] - 182s 195ms/step - loss: 0.0950


In [11]:
model.save_pretrained("gpt2-poetry-tf")
tokenizer.save_pretrained("gpt2-poetry-tf")

('gpt2-poetry-tf/tokenizer_config.json',
 'gpt2-poetry-tf/special_tokens_map.json',
 'gpt2-poetry-tf/vocab.json',
 'gpt2-poetry-tf/merges.txt',
 'gpt2-poetry-tf/added_tokens.json')

In [12]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
model = TFGPT2LMHeadModel.from_pretrained("gpt2-poetry-tf")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-poetry-tf")
prompt = "First you wanna go"
input_ids = tokenizer.encode(prompt, return_tensors='tf')

output = model.generate(
    input_ids,
    max_new_tokens=80,
    temperature=1.0,
    top_k=500,
    top_p=0.95,
    do_sample=True,
    no_repeat_ngram_size=3,
    pad_token_id=tokenizer.eos_token_id,
)

generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_poem)


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2-poetry-tf.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


First you wanna go to the left and you want to turn right (ooh)
